In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext ipython_unittest
%load_ext ipython_nose
%load_ext ipython_pytest
from IPython.display import HTML
import termios, fcntl, struct
fcntl.ioctl(1, termios.TIOCSWINSZ, struct.pack('hhhh', 57, 102, 0, 0))  # terminal width correction
HTML('''<link rel="stylesheet" href="eniram-theme/eniram-theme.css" type="text/css"></link>
        <script type="text/javascript" src="eniram-theme/rise-shortcuts.js"></script>''')

# Test parameterization
- run the same test scenario with different inputs
- avoid re-writing test setup/execution/teardown for each case

<br/>

## Challenges
- clarity of syntax
- labeling arguments
- how to identify tests from test runner output
- running single tests

## Parameterization methods and libraries
- unittest:
  - [metaclasses](http://stackoverflow.com/a/20870875/15770)
  - [load_tests](http://stackoverflow.com/a/23508426/15770)
  - [inheritance](http://bugs.python.org/msg151444)
  - [unittest.TestCase.subTest()](https://docs.python.org/3/library/unittest.html#distinguishing-test-iterations-using-subtests)
    (Python >= 3.4, unittest2)
- nose_parameterized
- @pytest.mark.parametrize()
- testdimensions (PoC from Eniram)
- lots of other methods on [Stack Overflow](http://stackoverflow.com/q/32899/15770)

## Example function to test
Let's pretend we just implemented the interpolation method in Pandas and want to write parameterized tests for it.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(19, 3))

import pandas as pd
import numpy as np

plt.subplot(131)
data = pd.Series([1.0, 2.0, np.nan, np.nan, 8.0, 13.0])
data.plot()
plt.subplot(132)
i = data.interpolate('spline', order=1) ; i.plot() ; i[2:4].plot(marker='o')
plt.subplot(133)
j = data.interpolate('spline', order=2) ; j.plot() ; j[2:4].plot(marker='o');

## Unittest: separate tests in a test case class
- pretty good if no setup/teardown needed
- <span style="color:green">&#x271a;</span> easy to identify tests from test output
- <span style="color:green">&#x271a;</span> can run single tests
- <span style="color:green">&#x271a;</span> no need to label parameters
- <span style="color:red">&#x26d4;</span> verbose; repeated code

In [ ]:
from unittest import TestCase
from numpy.testing import assert_allclose

In [ ]:
%%unittest

class InterpolateSplineTestCase(TestCase):
    data = pd.Series([1.0, 2.0, np.nan, np.nan, 8.0, 13.0])

    def test_interpolation_spline_order_1(self):
        result = self.data.interpolate('spline', order=1)   # (+) no need to label parameters
        assert_allclose(result[2:4], [4.814750, 7.077042])
        
    def test_interpolation_spline_order_2(self):            # (+) test method name as test description
        result = self.data.interpolate('spline', order=2)   # (-) repeated code
        assert_allclose(result[2:4], [2.852941, 5.13])


# Test cases as functions
- works with Nose and Pytest
- <span style="color:green">&#x271a;</span> easy to identify tests from test output
- <span style="color:green">&#x271a;</span> can run single tests
- <span style="color:green">&#x271a;</span> no need to label parameters
- <span style="color:red">&#x26d4;</span> repeated code

In [ ]:
%%nose -v --expand-tracebacks

data = pd.Series([1.0, 2.0, np.nan, np.nan, 8.0, 13.0])

def test_interpolation_spline_order_1():
    """Series.interpolate() with 1st order spline"""   # (+) add description for test case
    result = data.interpolate('spline', order=1)
    assert_allclose(result[2:4], [4.814750, 7.077042])

def test_interpolation_spline_order_2():
    """Series.interpolate() with 2nd order spline"""
    result = data.interpolate('spline', order=2)       # (+) no need to label parameters
    assert_allclose(result[2:4], [2.852941, 5.13])

# `TestCase.subTest()`
- <span style="color:green">&#x271a;</span> can identify test failures with `subTest(<keyword>=<value>, ...)`
- <span style="color:green">&#x271a;</span> can label parameters by defining tests as e.g. dicts
- <span style="color:red">&#x26d4;</span> can't run single tests
- <span style="color:red">&#x26d4;</span> boilerplate

In [ ]:
%%unittest

class CircumferenceTestCase(TestCase):
    data = pd.Series([1.0, 2.0, np.nan, np.nan, 8.0, 13.0])
    
    tests = [{'order': 1,                                 # (+) can label test parameters
              'expect': [4.814750, 7.077042]},
             {'order': 2,
              'expect': [2.852941, 5.13]}]
    
    def test_interpolation_spline(self):
        for arguments in self.tests:                      # (-) boilerplate
            with self.subTest(order=arguments['order']):  # (+) can add identification for tests
                result = self.data.interpolate('spline', order=arguments['order'])
                assert_allclose(result[2:4], arguments['expect'])

# Parameterize using `TestCase` inheritance
- <span style="color:green">&#x271a;</span> easy to identify tests from test outputUnittes
- <span style="color:green">&#x271a;</span> can run single test cases
- <span style="color:green">&#x271a;</span> easy to label parameters
- <span style="color:green">&#x271a;</span> can parameterize multiple tests in one test case class
- <span style="color:red">&#x26d4;</span> boilerplate

In [ ]:
%%unittest

class InterpolateSplineTestBase:
    data = pd.Series([1.0, 2.0, np.nan, np.nan, 8.0, 13.0])

    def test_interpolation_spline(self):                             # (+) could have multiple test methods
        result = self.data.interpolate('spline', order=self.order)
        assert_allclose(result[2:4], self.expect)
        
class InterpolateSplineOrder1(InterpolateSplineTestBase, TestCase):
    order = 1                                                        # (+) easy to label parameters
    expect = [4.814750, 7.077042]

class InterpolateSplineOrder2(InterpolateSplineTestBase, TestCase):  # (-) inheritance boilerplate
    order = 2
    expect = [2.852941, 5.13]

## Nose `yield` style parameterization
- <span style="color:green">&#x271a;</span> convenient e.g. when creating tests in (nested) `for` loops
- <span style="color:red">&#x26d4;</span> difficult to identify test failures by parameters
- <span style="color:red">&#x26d4;</span> can't label parameters
- <span style="color:red">&#x26d4;</span> can't run single tests
- <span style="color:red">&#x26d4;</span> yield syntax strange

In [ ]:
%%nose -v

def test_interpolate_with_yield():
    data = pd.Series([1.0, 2.0, np.nan, np.nan, 8.0, 13.0])  # (-) can't label parameters
    
    def check(order, expect):
        result = data.interpolate('spline', order=order)
        assert_allclose(result[2:4], expect)

    yield check, 1, [4.814750, 7.077042]                     # (+) strange syntax
    yield check, 2, [2.852941, 5.13]

# nose_parameterized
- a Nose helper available on PyPI
- parameterizes test class methods and functions
- <span style="color:green">&#x271a;</span> concise test code
- <span style="color:green">&#x271a;</span> customizable test description (coming up in 0.6)
- <span style="color:green">&#x271a;</span> can label parameters using `param()`
- <span style="color:red">&#x26d4;</span> hard to run single test cases (can't have docstring;
  find test ID in test output)
```
pip install nose-parameterized==0.5.0
```

## `@parameterized` with keyword arguments

In [ ]:
from nose_parameterized import parameterized, param

In [ ]:
%%nose -v

@parameterized([param(order=1,                         # (+) can label test parameters
                      expect=[4.814750, 7.077042]),
                param(order=2,
                      expect=[2.852941, 5.13])])
def test_interpolate_noseparameterized(order, expect):
    data = pd.Series([1.0, 2.0, np.nan, np.nan, 8.0, 13.0])
    result = data.interpolate('spline', order=order)
    assert_allclose(result[2:4], expect)

## `@parameterized` custom test descriptions

- will be available nose_parameterized 0.6
- <span style="color:green">&#x271a;</span> easy to identify tests
- <span style="color:red">&#x26d4;</span> test description goes in a separate documentation function
- <span style="color:red">&#x26d4;</span> not yet released
```
pip install \
https://github.com/wolever/nose-parameterized/archive/master.zip
```

In [ ]:
from nose_parameterized import param, parameterized

def interpolation_doc_func(func, num, param):          # (-) separate documentation function
    return ('Series.interpolate() with {description}'
            .format(**param.kwargs))

In [ ]:
%%nose -v

@parameterized(
    [param(description='1st order spline',              # (+) test description
           order=1,
           expect=[4.814750, 7.077042]),
     param(description='2nd order spline',
           order=2,
           expect=[2.852941, 5.13])],
    doc_func=interpolation_doc_func)
def test_interpolate_noseparameterized_doc_func(description, order, expect):
    data = pd.Series([1.0, 2.0, np.nan, np.nan, 8.0, 13.0])
    result = data.interpolate('spline', order=order)
    assert_allclose(result[2:4], expect)

## `@parameterized` subclass with docstring template
- <span style="color:green">&#x271a;</span> easy to identify tests
- <span style="color:green">&#x271a;</span> test description template in docstring
- <span style="color:red">&#x26d4;</span> nose_parameterized 0.6 not yet released
- <span style="color:red">&#x26d4;</span> custom implementation as a subclass of `@parameterized`

In [ ]:
from nose_parameterized.parameterized import default_doc_func

class parameterized_plus_description(parameterized):
    @staticmethod
    def _template_doc_func(func, num, param):                   # the template helper method
        if func.__doc__:
            return func.__doc__.format(**param.kwargs)
        else:
            return default_doc_func(func, num, param)    
    
    def __init__(self, input):                                  # override @parameterized()
        super().__init__(input, doc_func=self._template_doc_func)
        
    @classmethod
    def expand(cls, input, **kwargs):                           # override @parameterized.expand()
        return super().expand(input, doc_func=cls._template_doc_func, **kwargs)

In [ ]:
%%nose -v

@parameterized_plus_description(                   #  instead of @parameterized()
    [param(description='1st order spline',
           order=1,
           expect=[4.814750, 7.077042]),
     param(description='2nd order spline',
           order=2,
           expect=[2.852941, 5.13])])
def test_interpolate_noseparameterized(description, order, expect):
    """Series.interpolate() with {description}"""  # test description template as docstring
    data = pd.Series([1.0, 2.0, np.nan, np.nan, 8.0, 13.0])
    result = data.interpolate('spline', order=order)
    assert_allclose(result[2:4], expect)

## @pytest.mark.parametrize
- the built-in parameterization solution in pytest
- <span style="color:green">&#x271a;</span> comprehensive test output, tunable with ``--tb=``
- <span style="color:green">&#x271a;</span> can identify tests if `ids=` argument is used
- <span style="color:red">&#x26d4;</span> separate `ids=` argument to add descriptions to tests
- <span style="color:red">&#x26d4;</span> must enumerate argument names (due to Pytest fixtures)
- <span style="color:red">&#x26d4;</span> can't label parameters
- <span style="color:red">&#x26d4;</span> can't run single tests

## @pytest.mark.parametrize: example

In [ ]:
%%pytest -v --tb=short

import numpy as np, pandas as pd, pytest
from numpy.testing import assert_allclose

@pytest.mark.parametrize(
    'order,expect',                                # (-) must enumerate parameter names
    [(1, [4.814750, 7.077042]),                    # (-) can't label parameters
     (2, [2.852941, 5.13])],
    ids=['1st order spline', '2nd order spline'])  # (+/-) test descriptions as a separate list
def test_interpolate_pytest(order, expect):
    data = pd.Series([1.0, 2.0, np.nan, np.nan, 8.0, 13.0])
    result = data.interpolate('spline', order=order)
    assert_allclose(result[2:4], expect)

## @pytest.mark.parametrize: test description as an argument
- <span style="color:green">&#x271a;</span> test description together with test parameters
- <span style="color:green">&#x271a;</span> can identify tests
- <span style="color:red">&#x26d4;</span> function name and all arguments still visible in output

In [ ]:
%%pytest -v --tb=short

import numpy as np, pandas as pd, pytest
from numpy.testing import assert_allclose

@pytest.mark.parametrize(
    'description,order,expect',
    [('1st order spline', 1, [4.814750, 7.077042]),  # (+) test description as a parameter
     ('2nd order spline', 2, [2.852941, 5.13])])
def test_interpolate_pytest(description, order, expect):
    data = pd.Series([1.0, 2.0, np.nan, np.nan, 8.0, 13.0])
    result = data.interpolate('spline', order=order)
    assert_allclose(result[2:4], expect)

## `testdimensions` – multi-dimensional tests PoC
- visual test matrix for parameterizing in 2-D
- multiple matrices for 3-D and beyond
- built on `@pytest.mark.parametrize` – same limitations apply

In [ ]:
%%pytest -v --tb=short

import pandas as pd, numpy as np, pytest
from testdimensions import pytest_mark_dimensions

@pytest_mark_dimensions(
    'data,index,method,order,expect',
    """
    index = [0, 1, 2, 3, 4, 5]
    
                            1            2            3
    'spline'      [4.32,6.16]  [3.82,5.68]  [3.70,5.80]
    'polynomial'  [4.00,6.00]  [3.75,5.75]  [3.70,5.80]

    index = [0, 1, 3, 5, 8, 13]
    
                            1            2            3
    'spline'      [3.47,4.89]  [4.13,5.88]  [3.94,5.74]
    'polynomial'  [3.71,5.43]  [3.91,5.66]  [3.94,5.74]
    """,
    data=[1.0, 2.0, np.nan, np.nan, 8.0, 10.0])
def test_interpolate(data, index, method, order, expect):
    series = pd.Series(data, index=index)
    result = series.interpolate(method=method, order=order)
    expect_series = pd.Series(expect)
    np.testing.assert_allclose(result[2:4], expect_series, equal_nan=True, atol=1e-2)

In [ ]:
%%pytest -v --tb=short

import pandas as pd, numpy as np, pytest
from testdimensions import pytest_mark_dimensions

@pytest_mark_dimensions(
    'data,index,method,order,expect',
    """
    index = [0, 1, 2, 3, 4, 5]
    
                            1            2            3
    'spline'      [4.32,6.16]  [3.82,5.68]  [3.70,5.80]
    'polynomial'  [4.00,6.00]  [3.75,5.75]  [3.70,5.80]

    index = [0, 1, 3, 5, 8, 13]
    
                            1            2            3
    'spline'      [3.47,4.89]  [4.13,5.88]  [3.94,5.74]
    'polynomial'  [3.71,5.43]  [3.91,5.66]  [3.94,5.74]
    """,
    data=[1.0, 2.0, np.nan, np.nan, 8.0, 10.0])
def test_interpolate(data, index, method, order, expect):
    series = pd.Series(data, index=index)
    result = series.interpolate(method=method, order=order)
    expect_series = pd.Series(expect)
    np.testing.assert_allclose(result[2:4], expect_series, equal_nan=True, atol=1e-2)

The test case tested these 12 parameter combinations:

In [ ]:
plt.rc('figure', figsize=(19, 7)); subplot_idx = 0
for index in [list(range(6)), [0, 1, 3, 5, 8, 13]]:
    for method in ['spline', 'polynomial']:
        for order in range(1, 4):
            subplot_idx += 1
            plt.subplot(3, 4, subplot_idx)
            data = pd.Series([1.0, 2.0, np.nan, np.nan, 8.0, 13.0], index=index).interpolate(method, order=order)
            data.plot() ; data[2:4].plot(marker='o')

## `testdimensions` – try it out

https://github.com/EniramLtd/testdimensions

<img src="testdimensions.github.png" />